In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.models import resnet50
from torchvision.models import googlenet

# MNIST Dataset Setup

In [ ]:
"""Downloading the MNIST dataset"""

train_loader = torch.utils.data.DataLoader( torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize((0.1307,), (0.3081,))])),
                            batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))])),
                            batch_size=batch_size_test, shuffle=True)

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

example_data.shape

In [ ]:
"""Visualizing some of the ground truth image."""

import matplotlib.pyplot as plt
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

#ResNet

In [ ]:
"""Initialize the training and testing parameters."""

n_epochs = 5
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = True
torch.manual_seed(random_seed)

In [6]:
"""Initializing model with output for 10 classes and converting model to accept """

model = resnet50(num_classes=10)  #number of classes in MNIST is 10. This modifies the output features of the last FC layer.
#changing to single channel
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

#Using SGD optimizer with LR=0.01 and Momentum=0.5
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
#optimizer = optim.Adam(model.parameters(), lr=0.01)

criterion = nn.CrossEntropyLoss()

#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

if torch.cuda.is_available():
    model = model.cuda()      #push the parameters to default device.
    criterion = criterion.cuda()

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
test_acc1 = []



In [7]:
"""Training model function"""

def train(epoch):
  model.train()

  for batch_idx, (data, target) in enumerate(train_loader):
    #if GPU is available, move data and target to GPU
    if torch.cuda.is_available():
      data=data.cuda()
      target=target.cuda()
    
    #compute output and loss.    
    output = model(data)
    loss = criterion(output, target)

    #backward and update model.

    #In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    #This is convenient while training RNNs. So, the default action is to accumulate (i.e. sum) the gradients on every loss.backward() call.
    #Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.
    #Else the gradient would point in some other direction than the intended direction towards the minimum (or maximum, in case of maximization objectives).

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      
      train_losses.append(loss.item())
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

      torch.save(model.state_dict(), '/resnetmodel.pth')
      torch.save(optimizer.state_dict(), '/optimizer.pth')


In [ ]:
"""Testing model function"""
def test():
  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:
      if torch.cuda.is_available():
        data=data.cuda()
        target=target.cuda()

      output = model(data)
      test_loss += criterion(output, target).item()

      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()

  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  test_acc = 100 * correct / len(test_loader.dataset)
  test_acc1.append(test_acc)

  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [ ]:
"""Training and testing results at each epoch"""

epochs=[]
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()
  epochs.append(epoch)

In [ ]:
"""Ploting loss curve """

fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.legend(['Train Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('loss')
fig

In [ ]:
"""Plotting accuracy curve"""

fig = plt.figure()
plt.plot(epochs, test_acc1, color='blue')
plt.legend(['Train Accuracy'], loc='upper right')
plt.xlabel('number of epochs')
plt.ylabel('Accuracy %')
fig

In [ ]:
"""Visualizing predictions"""

with torch.no_grad():
  example_data=example_data.cuda()
  output = model(example_data)

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Prediction: {}".format(
    output.data.max(1, keepdim=True)[1][i].item()))
  plt.xticks([])
  plt.yticks([])
fig

In [10]:
"""### GoogleNet"""

model2 = googlenet(num_classes=10)
model2.conv1.conv = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model2=model2.cuda()

optimizer = optim.SGD(model2.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses2 = []
train_counter2 = []
test_losses2 = []
test_counter2 = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
test_acc2 = []

/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:79: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)


In [11]:
"""Training function"""

def train(epoch):
  model2.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    data=data.cuda()
    target=target.cuda()
    output = model2(data)
    loss = criterion(output.logits, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses2.append(loss.item())
      train_counter2.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(model2.state_dict(), '/googlenetmodel.pth')
      torch.save(optimizer.state_dict(), '/optimizer.pth')

"""Testing function"""

def test():
  model2.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data=data.cuda()
      target=target.cuda()
      output = model2(data)
      test_loss += criterion(output, target).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses2.append(test_loss)
  test_acc = 100 * correct / len(test_loader.dataset)
  test_acc2.append(test_acc)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
"""Visualizing loss and accuracy at each epoch"""

epochs=[]
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()
  epochs.append(epoch)

In [ ]:
"""Plotting training loss"""

fig = plt.figure()
plt.plot(train_counter2, train_losses2, color='blue')
plt.plot(train_counter2, train_losses, color='green')
plt.legend(['GoogleNet Train Loss','ResNet50 Train Loss' ], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('loss')
fig

In [ ]:
"""Plotting testing accuracy"""

fig = plt.figure()
plt.plot(epochs, test_acc1, color='blue')
plt.plot(epochs, test_acc2, color='green')
plt.legend(['Test Accuracy (ResNet50)','Test Accuracy (GoogleNet)'], loc='lower right')
plt.xlabel('number of epochs')
plt.ylabel('Accuracy %')
fig

In [ ]:
"""Visualizing predicted data"""

with torch.no_grad():
  example_data = example_data.cuda()
  output = model(example_data)

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Prediction: {}".format(
    output.data.max(1, keepdim=True)[1][i].item()))
  plt.xticks([])
  plt.yticks([])
fig